In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from sklearn.metrics import f1_score

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor()
])

In [ ]:
dataset_path = "/content/drive/MyDrive/dataset"

full_dataset = datasets.ImageFolder(
    root=dataset_path,
    transform=transform
)

print("Total images:", len(full_dataset))
print("Number of classes:", len(full_dataset.classes))

Total images: 9148
Number of classes: 102


In [ ]:
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(
    full_dataset,
    [train_size, val_size]
)

print("Train size:", len(train_dataset))
print("Val size:", len(val_dataset))

Train size: 7318
Val size: 1830


In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0   # <-- CHANGE THIS
)

val_loader = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)

In [ ]:
model = models.resnet18(pretrained=True)

num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(full_dataset.classes))

model = model.to(device)

print("ResNet ready.")

ResNet ready.


In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(
    model.parameters(),
    lr=0.0001
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
num_epochs = 10
print_interval = 1  # 👈 change this if you want

for epoch in range(num_epochs):
    print(f"\n===== Epoch {epoch+1}/{num_epochs} =====")

    # -------- TRAIN --------
    model.train()
    correct = 0
    total = 0
    running_loss = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        # 🔥 Print every 100 batches
        if (batch_idx + 1) % print_interval == 0:
            avg_loss = running_loss / print_interval
            print(f"Batch [{batch_idx+1}/{len(train_loader)}] "
                  f"Avg Loss: {avg_loss:.4f}")
            running_loss = 0

    train_acc = 100 * correct / total

    # -------- VALIDATION --------
    model.eval()
    val_correct = 0
    val_total = 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)

            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_acc = 100 * val_correct / val_total
    val_f1 = f1_score(all_labels, all_preds, average='macro')

    print(f"\nEpoch {epoch+1} Summary:")
    print(f"Train Accuracy: {train_acc:.2f}%")
    print(f"Validation Accuracy: {val_acc:.2f}%")
    print(f"Validation F1-score (macro): {val_f1:.4f}")


===== Epoch 1/10 =====
Batch [1/229] Avg Loss: 1.7560
Batch [2/229] Avg Loss: 1.5199
Batch [3/229] Avg Loss: 1.5702
Batch [4/229] Avg Loss: 1.5926
Batch [5/229] Avg Loss: 1.7427
Batch [6/229] Avg Loss: 1.7558
Batch [7/229] Avg Loss: 2.1463
Batch [8/229] Avg Loss: 2.0018
Batch [9/229] Avg Loss: 1.6914
Batch [10/229] Avg Loss: 1.3878
Batch [11/229] Avg Loss: 2.0633
Batch [12/229] Avg Loss: 1.1749
Batch [13/229] Avg Loss: 1.7714
Batch [14/229] Avg Loss: 1.4348
Batch [15/229] Avg Loss: 1.7931
Batch [16/229] Avg Loss: 1.5965
Batch [17/229] Avg Loss: 1.8184
Batch [18/229] Avg Loss: 1.8854
Batch [19/229] Avg Loss: 1.6834
Batch [20/229] Avg Loss: 2.0175
Batch [21/229] Avg Loss: 1.4014
Batch [22/229] Avg Loss: 1.3226
Batch [23/229] Avg Loss: 1.2379
Batch [24/229] Avg Loss: 1.7459
Batch [25/229] Avg Loss: 1.3290
Batch [26/229] Avg Loss: 1.5668
Batch [27/229] Avg Loss: 1.2972
Batch [28/229] Avg Loss: 1.6803
Batch [29/229] Avg Loss: 1.5630
Batch [30/229] Avg Loss: 1.5931
Batch [31/229] Avg Loss: 